In [ ]:
import ee
import time

ee.Authenticate()
ee.Initialize(project='dse-staff')

GSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select("max_extent")
HM_IMAGE = ee.ImageCollection('CSP/HM/GlobalHumanModification').mean()
MODIS = ee.ImageCollection('MODIS/006/MOD09A1')

In [ ]:
samples = ee.FeatureCollection('projects/dse-staff/assets/transects_final')

In [ ]:
water_results = GSW.reduceRegions(
    collection=samples,
    reducer=ee.Reducer.max().setOutputs(['water_max_extent']),
    scale=30,  # used 30m bc 500m misses many water pts
    tileScale=8
)

In [ ]:
hm_results = HM_IMAGE.reduceRegions(
    collection=water_results,
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True).combine(ee.Reducer.median(), '', True)
        .setOutputs(['hm_mean', 'hm_std', 'hm_med']),
    scale=500,
    tileScale=8
)

In [ ]:
# Process multiple years of MODIS gradient data with task queue management
years = list(range(2001, 2002)) 
max_concurrent_tasks = 15
submitted_tasks = []

def check_task_status():
    """Check status of submitted tasks and remove completed ones"""
    global submitted_tasks
    active_tasks = []
    for task_obj, year in submitted_tasks:
        task_status = task_obj.status()
        if task_status['state'] in ['COMPLETED', 'FAILED', 'CANCELLED']:
            print(f"Task {year} {task_status['state']}")
        else:
            active_tasks.append((task_obj, year))
    submitted_tasks = active_tasks
    return len(submitted_tasks)

for i, year in enumerate(years):
    # Wait if we have too many active tasks
    while check_task_status() >= max_concurrent_tasks:
        print(f"Waiting... {len(submitted_tasks)} tasks active")
        time.sleep(30)  # Check every 30 seconds
    
    print(f"Processing year {year} ({i+1}/{len(years)})...")
    
    # Get MODIS and calculate NDVI
    modis = MODIS.filterDate(f'{year}-01-01', f'{year}-12-31') \
        .max() \
        .select(['sur_refl_b01', 'sur_refl_b02'])  
    
    # Calculate NDVI
    ndvi = modis.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('ndvi').select('ndvi')
    
    # Calculate gradient 
    grad = ndvi.gradient()
    magnitude = grad.expression('sqrt(x*x + y*y)', {'x': grad.select('x'), 'y': grad.select('y')})

    # Reduce with explicit CRS and scale matching MODIS
    final_results = magnitude.reduceRegions(
        collection=hm_results,
        reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True).combine(ee.Reducer.count(), '', True)
            .setOutputs(['grad_mean', 'grad_std', 'grad_count']),
        scale=500,   
        tileScale=8
    )
    
    # Export results and track task
    export_task = ee.batch.Export.table.toDrive(
        collection=final_results,
        description=f'ndvi_results_{year}',
        folder='pa_results_20251204',  # This will create/use a 'results' folder in your Google Drive
        fileNamePrefix=f'ndvi_results_{year}',
        fileFormat='CSV',
        selectors=['WDPA_PID', 'ORIG_NAME', 'GOV_TYPE', 'OWN_TYPE',
        'STATUS_YR', 'IUCN_CAT', 'GIS_AREA', 'CONS_OBJ', 'DESIG', 'DESIG_ENG',
        'DESIG_TYPE', 'GIS_M_AREA', 'INT_CRIT', 'ISO3', 'MANG_AUTH',
        'MANG_PLAN', 'MARINE', 'METADATAID', 'NAME', 'NO_TAKE',
        'NO_TK_AREA', 'PARENT_ISO', 'PA_DEF', 'REP_AREA', 'REP_M_AREA',
        'STATUS', 'SUB_LOC', 'SUPP_INFO', 'VERIF', 'WDPAID','BIOME_NAME',
        'PA_RATIO', 'AREA_DISSO', 'PERIMETER',
        'transectID', 'point_position', 'water_max_extent',
        'hm_mean', 'hm_std', 'hm_med', 
        'grad_mean', 'grad_std', 'grad_count']
    )
    export_task.start()
    submitted_tasks.append((export_task, year))
    print(f"Export task started: {export_task.id} for {year}")

# Wait for remaining tasks to complete
print("Waiting for remaining tasks to complete...")
while check_task_status() > 0:
    print(f"Still waiting for {len(submitted_tasks)} tasks...")
    time.sleep(30)

print("All export tasks completed!")